In [1]:
import os
import pandas as pd
import numpy as np
import pickle

In [2]:
import xenaPython as xena

/home/pdutta/anaconda3/envs/dgcca/lib/python3.7/site-packages/xenaPython/__init__.py:110: FutureWarning: Possible nested set at position 7
  re.sub(r"^[^[]+[[]([^]]*)[]].*$", r"\1", query, flags=re.DOTALL))


In [3]:
xena.PUBLIC_HUBS

{'publicHub': 'https://ucscpublic.xenahubs.net',
 'tcgaHub': 'https://tcga.xenahubs.net',
 'icgcHub': 'https://icgc.xenahubs.net',
 'toilHub': 'https://toil.xenahubs.net',
 'pcawgHub': 'https://pcawg.xenahubs.net',
 'singlecellHub': 'https://singlecell.xenahubs.net',
 'pancanAtlasHub': 'https://pancanatlas.xenahubs.net',
 'treehouseHub': 'https://xena.treehouse.gi.ucsc.edu',
 'gdcHub': 'https://gdc.xenahubs.net',
 'reference': 'https://reference.xenahubs.net'}

In [4]:
CANCER= "OV"
host = xena.PUBLIC_HUBS['tcgaHub']
survival_dataset = 'survival/{}_survival.txt'.format(CANCER)
mirna_view = "TCGA.{}.sampleMap/miRNA_HiSeq_gene".format(CANCER)
methyl_view1 = "TCGA.{}.sampleMap/HumanMethylation27".format(CANCER)
methyl_view2 = "TCGA.{}.sampleMap/HumanMethylation450".format(CANCER)
copy_number_view = "TCGA.{}.sampleMap/Gistic2_CopyNumber_Gistic2_all_data_by_genes".format(CANCER)

In [5]:
output_path = "/home/pdutta/Data/DGCCA/TCGA_DATA/TCGA_{}/".format(CANCER)

In [6]:
survival_samples = xena.dataset_samples (host, survival_dataset, None)

In [7]:
output_path

'/home/pdutta/Data/DGCCA/TCGA_DATA/TCGA_OV/'

In [8]:
len(survival_samples)

604

In [9]:
result = set()
for dataset in [methyl_view1, methyl_view2]:
    print(dataset)
    samples = xena.dataset_samples (host, dataset, None)
    features = xena.dataset_field(host, dataset)
    result = result.union(set(samples))
    list1 = xena.dataset_fetch(host, dataset, samples, features)
    list_without_nan= [x for x in list1 if 'NaN' not in x]
    tran_list = list(map(list, zip(*list_without_nan)))
    feature = np.array([np.array(xi) for xi in tran_list])
    print(feature.shape)
print(len(result))

TCGA.OV.sampleMap/HumanMethylation27
(616, 21667)
TCGA.OV.sampleMap/HumanMethylation450
(10, 395576)
626


In [10]:
result = set()
view_list = [mirna_view, copy_number_view, methyl_view1]
feature_lists =[]
for dataset in view_list:
    print(dataset)
    samples = xena.dataset_samples (host, dataset, None)
    features = xena.dataset_field(host, dataset)
    result = result.union(set(samples))
    list1 = xena.dataset_fetch(host, dataset, samples, features)
    list_without_nan= [x for x in list1 if 'NaN' not in x]
    tran_list = list(map(list, zip(*list_without_nan)))
    feature = np.array([np.array(xi) for xi in tran_list])
    print(feature.shape, len(samples))
    #print(globals()['df_%s_features' % dataset])
    temp_df = pd.DataFrame(tran_list)
    temp_df['sample'] = samples
    temp_df = temp_df.set_index(['sample'])
    feature_lists.append(temp_df)
print(len(result))

TCGA.OV.sampleMap/miRNA_HiSeq_gene
(485, 211) 485
TCGA.OV.sampleMap/Gistic2_CopyNumber_Gistic2_all_data_by_genes
(579, 24777) 579
TCGA.OV.sampleMap/HumanMethylation27
(616, 21667) 616
627


In [11]:
common_samples = set(survival_samples).intersection(result)
print(len(common_samples))
df_survival = pd.DataFrame(survival_samples, columns = ['sample'])
df_survival.to_csv(output_path+"id_mapping.tsv", sep= "\t", index=False)

604


In [18]:
feature_lists[0].head()

,0,1,2,3,4,5,6,7,8,9,...,201,202,203,204,205,206,207,208,209,210
sample,,,,,,,,,,,,,,,,,,,,,
TCGA-61-1910-01,17.26,16.40,14.40,10.250,12.21,13.02,6.837,8.848,8.019,12.48,...,4.221,3.247,2.821,4.643,2.913,3.458,4.228,4.060,6.544,0
TCGA-61-1728-01,17.28,17.37,15.04,8.672,11.93,11.57,6.433,7.059,8.146,12.36,...,5.053,5.700,2.671,6.325,3.496,4.338,3.978,4.775,4.018,1
TCGA-31-1953-01,16.98,15.94,16.02,9.540,12.82,12.88,8.403,8.294,8.100,14.22,...,4.949,5.620,4.836,6.879,4.714,4.700,4.849,2.944,4.862,2
TCGA-31-1951-01,17.05,16.89,14.50,9.496,13.40,11.57,6.824,8.038,8.358,13.28,...,4.614,5.085,3.820,6.469,7.934,4.402,5.470,4.177,4.921,3
TCGA-24-1552-01,16.68,16.56,14.94,9.015,11.07,10.91,7.762,6.741,7.770,11.35,...,4.567,5.970,4.801,6.460,4.932,3.221,5.703,4.260,2.470,4


In [12]:
df_survival = df_survival.set_index('sample')

In [17]:
for view_name, view_feature in zip(['miRNA','CopyNumber', 'Methyl'], feature_lists):
    print(view_name, view_feature.shape)
    df_temp_join = df_survival.join(view_feature).fillna(0)
    print(df_temp_join)
    #np.save(id_mapping_path+view_name+"_features", df_temp_join.to_numpy())
    file = open(output_path+view_name+"_features.pickle", 'wb')
    pickle.dump(df_temp_join.to_numpy(), file)
    print(df_temp_join.shape)

miRNA (485, 211)
              sample    0    1    2    3    4    5    6    7    8  ...  201  \
0    TCGA-04-1331-01  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
1    TCGA-04-1332-01  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
2    TCGA-04-1335-01  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
3    TCGA-04-1336-01  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
4    TCGA-04-1337-01  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
..               ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
599  TCGA-61-2614-01  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
600  TCGA-OY-A56P-01  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
601  TCGA-OY-A56Q-01  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
602  TCGA-VG-A8LO-01  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
603  TCGA-WR-A838-01  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   

     202  203  204  205  206  207 

In [13]:
for file in os.listdir(output_path):
    if file.endswith('.pickle'):
        print (file)
        with open(output_path+file, 'rb') as f:
            x = pickle.load(f)
        print(x.shape)
        print(x)

CopyNumber_features.pickle
(604, 24777)
[[ 0.014 -0.79   0.04  ...  0.074  0.074 -0.765]
 [ 0.4   -0.373  0.37  ...  0.309  0.309  0.353]
 [-0.767 -0.812  0.748 ... -0.814 -0.814 -0.781]
 ...
 [ 0.016 -1.178  0.211 ...  0.016  0.016 -0.381]
 [ 0.058 -0.916  1.761 ...  0.079  0.079  0.734]
 [-0.381 -0.503 -0.437 ... -0.381 -0.381 -0.293]]
Methyl_features.pickle
(604, 21667)
[[9.221e-01 2.940e-02 8.248e-01 ... 1.670e-02 2.020e-02 5.770e+02]
 [4.648e-01 3.450e-02 4.201e-01 ... 2.030e-02 3.700e-02 2.600e+02]
 [9.043e-01 5.520e-02 6.690e-01 ... 2.570e-02 1.650e-02 1.110e+02]
 ...
 [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]]
miRNA_features.pickle
(604, 211)
[[ 16.98    16.98    15.08   ...   3.118    2.43   454.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]


In [14]:
df_survival

,sample
0,TCGA-04-1331-01
1,TCGA-04-1332-01
2,TCGA-04-1335-01
3,TCGA-04-1336-01
4,TCGA-04-1337-01
...,...
599,TCGA-61-2614-01
600,TCGA-OY-A56P-01
601,TCGA-OY-A56Q-01
602,TCGA-VG-A8LO-01
